In [ ]:
import torch
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
%matplotlib inline
from torch.utils.data import Dataset, DataLoader
from data_provider.data_factory import data_provider

from tqdm.notebook import tqdm

from sklearn.metrics import r2_score, mean_squared_error


from autoformer_inference import Autoformer,time_features

In [ ]:
setting='qber_96_48_60_loginf_t_non_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'
# setting='qber_96_48_60_loginf_t_non_outliers_pred_3_Autoformer_custom_ftMS_sl96_ll48_pl3_dm512_nh8_el6_dl8_df2048_fc1_ebtimeF_dtFalse_Exp_0'
# setting='qber_96_48_60_loginf_t_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'

path=f'./checkpoints/{setting}/checkpoint.pth'

seq_len=96
label_len=48
# pred_len=1
pred_len=1

model = Autoformer(
        dropout=0.05,
        freq='t',
        seq_len=seq_len,
        label_len=label_len,
        pred_len=pred_len,
        e_layers=6,
        d_layers=6,
        # d_layers=8,
        n_heads=8,
        factor=1,
        enc_in=6,
        dec_in=6,
        c_out=6,
        d_model=512,
        d_ff=2048,
        moving_avg=25,
        output_attention=True,
        embed='timeF',
        activation='gelu',)

model.load_state_dict(torch.load(path))

device='cuda'
model.to(device)

In [ ]:
setting='qber_96_12_60_loginf_t_non_outliers_Autoformer_custom_ftMS_sl96_ll12_pl1_dm512_nh8_el6_dl8_df2048_fc1_ebtimeF_dtFalse_Exp_0'
path=f'./checkpoints/{setting}/checkpoint.pth'

seq_len=96
label_len=12
pred_len=1

model = Autoformer(
    dropout=0.05,
    freq='t',
    seq_len=seq_len,
    label_len=label_len,
    pred_len=pred_len,
    e_layers=6,
    d_layers=8,
    n_heads=8,
    factor=1,
    enc_in=6,
    dec_in=6,
    c_out=6,
    d_model=512,
    d_ff=2048,
    moving_avg=25,
    output_attention=True,
    embed='timeF',
    activation='gelu',)

model.load_state_dict(torch.load(path))

device='cuda'
model.to(device)


In [ ]:
def preprocess_autoformer(df_raw, seq_len, label_len,pred_len, device, embed='timeF', target='e_mu_current',freq='t', step=60):

    timeenc = 0 if embed != 'timeF' else 1

    cols = list(df_raw.columns)
    cols.remove(target)
    cols.remove('date')

    df_raw = df_raw[['date'] + cols + [target]]
    border1 = len(df_raw) - seq_len
    border2 = len(df_raw)

    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]

    data = df_data.values

    tmp_stamp = df_raw[['date']][border1:border2]
    tmp_stamp['date'] = pd.to_datetime(tmp_stamp.date)
    pred_dates = pd.date_range(tmp_stamp.date.values[-1], periods=pred_len + 1, freq=freq)
    
    df_stamp = pd.DataFrame(columns=['date'])
    df_stamp.date = list(tmp_stamp.date.values) + list(pred_dates[1:])

    if timeenc == 0:
        df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
        df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
        df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
        df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
        data_stamp = df_stamp.drop(['date'], 1).values
    elif timeenc == 1:
        data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=freq)
        data_stamp = data_stamp.transpose(1, 0)

    data_x = data[border1:border2]
    data_y = data[border1:border2]

    s_begin = 0
    s_end = s_begin + seq_len
    r_begin = s_end - label_len
    r_end = r_begin + label_len + pred_len

    seq_x = data_x[s_begin:s_end]
    seq_y = data_y[r_begin:r_begin + label_len]
    seq_x_mark = data_stamp[s_begin:s_end]
    seq_y_mark = data_stamp[r_begin:r_end]

    batch_x = torch.FloatTensor(seq_x).unsqueeze(0).to(device)
    batch_y = torch.FloatTensor(seq_y).unsqueeze(0).to(device)
    batch_x_mark = torch.FloatTensor(seq_x_mark).unsqueeze(0).to(device)
    batch_y_mark = torch.FloatTensor(seq_y_mark).unsqueeze(0).to(device)

    dec_inp = torch.zeros_like(batch_y[:, -pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :label_len, :], dec_inp], dim=1).float().to(device)

    return batch_x, batch_x_mark, dec_inp, batch_y_mark

def autoformer_predict(model, df_tmp, pred_len=3, f_dim=-1,seq_len=96,label_len=48, output_attention=True):

    batch_x, batch_x_mark, dec_inp, batch_y_mark = preprocess_autoformer(df_tmp,seq_len, label_len,pred_len, device=device)

    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

    if output_attention:
        outputs = outputs[0]

    outputs = outputs[:, -pred_len:, f_dim:]
    pred = outputs.detach().cpu().numpy()[0]

    return pred[-pred_len:]

In [ ]:
df = pd.read_csv('datasets/non_outliers_step=60_log_inf_test=0.1.csv')
# df = pd.read_csv('datasets/outliers_step=60_log_inf_test=0.1.csv')

In [ ]:
model.eval()

preds = []
trues = []

with torch.no_grad():
    for i  in tqdm(range(len(df)-seq_len)):
        
        df_tmp=df.iloc[i:i+seq_len]

        pred = autoformer_predict(model, df_tmp,pred_len=pred_len)[0]
        true = df['e_mu_current'].iloc[i+seq_len+pred_len-2]
        
        preds.append(pred)
        trues.append(true)


In [ ]:
setting='qber_96_12_60_loginf_t_non_outliers_Autoformer_custom_ftMS_sl96_ll12_pl1_dm512_nh8_el6_dl8_df2048_fc1_ebtimeF_dtFalse_Exp_0'

print('R^2:',r2_score(np.exp(trues), np.exp(preds)))
print('MSE:',mean_squared_error(np.exp(trues), np.exp(preds)))

In [ ]:
setting='qber_96_48_60_loginf_t_non_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'

print('R^2:',r2_score(np.exp(trues), np.exp(preds)))
print('MSE:',mean_squared_error(np.exp(trues), np.exp(preds)))

In [ ]:
trues=np.array(trues)
preds=np.array(preds)
plt.figure(figsize=(20,5))

plt.plot(np.exp(trues)[:300])
# plt.plot(np.exp(df['e_mu_current'].iloc[seq_len:326+seq_len])[:300])
plt.plot(np.exp(preds)[:300])
plt.legend(['true', 'predicted'])
plt.show()

In [ ]:
preds

In [ ]:
trues=np.array(trues)
preds=np.array(preds)
plt.figure(figsize=(15,5))

plt.plot(np.exp(trues))
plt.plot(np.exp(preds))
plt.legend(['true', 'predicted'])
plt.show()

In [ ]:
trues=np.array(trues)
preds=np.array(preds)
plt.figure(figsize=(15,5))

plt.plot(np.exp(trues))
plt.plot(np.exp(preds))
plt.legend(['true', 'predicted'])
plt.show()